# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [1]:
!pip install selenium
# chrome -> chromedriver
# firefox -> geckodriver

  Using cached selenium-4.5.0-py3-none-any.whl (995 kB)
  Using cached trio-0.22.0-py3-none-any.whl (384 kB)
  Using cached trio_websocket-0.9.2-py3-none-any.whl (16 kB)
  Using cached exceptiongroup-1.0.0rc9-py3-none-any.whl (12 kB)
  Using cached outcome-1.2.0-py2.py3-none-any.whl (9.7 kB)
  Using cached async_generator-1.10-py3-none-any.whl (18 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)


In [11]:
from selenium import webdriver # Permite criar o navegador
from selenium.webdriver.common.keys import Keys # Permite escrever no navegador
from selenium.webdriver.common.by import By # Permite selecionar itens no navegador

# Passo 1: Pegar a cotação do dolar
#Abrir no navegador
navegador = webdriver.Chrome()
#Entrar no google
navegador.get("https://www.google.com/")
#Pesquisar no google por cotação dolar
navegador.find_element("xpath", '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dolar")
navegador.find_element("xpath", '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
#Pegar a cotação que está no google
cotacao_dolar = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar +" R$")

# Passo 2: Pegar a cotação do euro
#Entrar no google
navegador.get("https://www.google.com/")
#Pesquisar no google por cotação euro
navegador.find_element("xpath", '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
navegador.find_element("xpath", '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
#Pegar a cotação que está no google
cotacao_euro = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro +" R$")

# Passo 3: Pegar a cotação do ouro
#Entrar no site do melhor cambio
navegador.get("https://www.melhorcambio.com/ouro-hoje")
#Pegar a cotação
cotacao_ouro = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro +" R$ por grama")
#Fechar o navegador
navegador.quit()

# Passo 5: Exportar a base de preços atualizado

5.2222 R$
5.111724359 R$
288.92 R$ por grama


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [14]:
# Passo 4: Atualizar a base de preços (atualizando o preço de compra e o de venda)
import pandas as pd
tabela = pd.read_excel("Produtos.xlsx") # Encontra a tabela no computador
display(tabela) #Mostra a tabela

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [15]:
#Atualizar a coluna de cotação
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

#Atualizar o preço de compra = preço original * cotação
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

#Atualizar a coluna de preço de venda = preço de compra * margem
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.222200,5222.147778,1.40,7311.006889
1,Carro Renault,4500.00,Euro,5.111724,23002.759615,2.00,46005.519231
2,Notebook Dell,899.99,Dólar,5.222200,4699.927778,1.70,7989.877223
3,IPhone,799.00,Dólar,5.222200,4172.537800,1.70,7093.314260
4,Carro Fiat,3000.00,Euro,5.111724,15335.173077,1.90,29136.828846
5,Celular Xiaomi,480.48,Dólar,5.222200,2509.162656,2.00,5018.325312
6,Joia 20g,20.00,Ouro,288.920000,5778.400000,1.15,6645.160000


### Agora vamos exportar a nova base de preços atualizada

In [17]:
# Exportar a base de dados autalizada
tabela.to_excel("Produtos-novo.xlsx", index=False)

AttributeError: 'DataFrame' object has no attribute 'to_odf'